In [19]:
import intake
import numpy as np
import pandas as pd

cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [17]:
iids = []
test = []
source_ids = ['ACCESS-CM2',
                 'ACCESS-ESM1-5',
                 'AWI-ESM-1-1-LR',
                 'CESM2',
                 'CESM2-FV2',
                 'CESM2-WACCM',
                 'CESM2-WACCM-FV2',
                 'CMCC-CM2-SR5',
                 'CMCC-ESM2',
                 'E3SM-1-0',
                 'EC-Earth3',
                 'EC-Earth3-AerChem',
                 'EC-Earth3-CC',
                 'EC-Earth3-Veg-LR',
                 'GFDL-CM4',
                 'GFDL-ESM4',
                 'GISS-E2-1-H',
                 'IPSL-CM6A-LR',
                 'IPSL-CM6A-LR-INCA',
                 'KIOST-ESM',
                 'MIROC6',
                 'MPI-ESM-1-2-HAM',
                 'MPI-ESM1-2-HR',
                 'MPI-ESM1-2-LR',
                 'SAM0-UNICON']

for source_id in source_ids:
    siv_query = col.search(
        experiment_id=['historical'], 
        table_id=['SImon'],
        source_id=source_id,
        variable_id=['siv'])
    
    siu_query = col.search(
        experiment_id=['historical'], 
        table_id=['SImon'],
        source_id=source_id,
        variable_id=['siu'])
    
    
    for row, data in siu_query.df.iterrows():
        member_id = data.member_id
        if member_id not in siv_query.df['member_id'].values:
            fname = data.zstore.replace('gs://cmip6/', '').replace('/', '.').replace('siu', 'siv')
            fname = fname[:-1]
            if fname not in iids:
                iids.append(fname)
        else:
            fname = data.zstore.replace('gs://cmip6/', '').replace('/', '.').replace('siu', 'siv')
            fname = fname[:-1]
            test.append(fname)
            
    for row, data in siv_query.df.iterrows():
        member_id = data.member_id
        if member_id not in siu_query.df['member_id'].values:
            fname = data.zstore.replace('gs://cmip6/', '').replace('/', '.').replace('siv', 'siu')
            fname = fname[:-1]
            if fname not in iids:
                iids.append(fname)

In [18]:
# CESM2 siv
# I found non-concatenated files for r8, r9, r10, and concatenated files for r3 and r2
# For CESM2-WACCM-FV2 the SIV files were missing for r2 and r3
iids

['s3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2.historical.r3i1p1f1.SImon.siv.gn.v20190308',
 's3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2.historical.r2i1p1f1.SImon.siv.gn.v20190308',
 's3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2.historical.r8i1p1f1.SImon.siv.gn.v20190311',
 's3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2.historical.r9i1p1f1.SImon.siv.gn.v20190311',
 's3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2.historical.r10i1p1f1.SImon.siv.gn.v20190313',
 's3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2-WACCM-FV2.historical.r3i1p1f1.SImon.siv.gn.v20200226',
 's3:..cmip6-pds.CMIP6.CMIP.NCAR.CESM2-WACCM-FV2.historical.r2i1p1f1.SImon.siv.gn.v20200226',
 's3:..cmip6-pds.CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r14i1p1f1.SImon.siv.gn.v20180803',
 's3:..cmip6-pds.CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r18i1p1f1.SImon.siv.gn.v20180803',
 's3:..cmip6-pds.CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r15i1p1f1.SImon.siv.gn.v20180803',
 's3:..cmip6-pds.CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r30i1p1f1.SImon.siv.gn.v20180803',
 's3:..cmip6-pds.CMIP6.

In [11]:
import intake

def zstore_to_iid(zstore: str):
    # this is a bit whacky to account for the different way of storing old/new stores
    iid =  '.'.join(zstore.replace('gs://','').replace('.zarr','').replace('.','/').split('/')[-11:-1])
    if not iid.startswith('CMIP6'):
        iid =  '.'.join(zstore.replace('gs://','').replace('.zarr','').replace('.','/').split('/')[-10:])
    return iid

def search_iids(col_url:str):
    col = intake.open_esm_datastore(col_url)
    iids_all= [zstore_to_iid(z) for z in col.df['zstore'].tolist()]
    return [iid for iid in iids_all if iid in iids_requested]


iids_requested = test

url_dict = {
    'qc':"https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog.json",
    'non-qc':"https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog_noqc.json",
    'retracted':"https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog_retracted.json"
}

iids_found = []
for catalog,url in url_dict.items():
    iids = search_iids(url)
    iids_found.extend(iids)
    print(f"Found in {catalog=}: {iids=}\n")

missing_iids = list(set(iids_requested) - set(iids_found))
print(f"\n\nStill missing {len(missing_iids)} of {len(iids_requested)}: \n{missing_iids=}")

Found in catalog='qc': iids=['CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r5i1p5f1.SImon.siv.gn.v20190905', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r3i1p5f1.SImon.siv.gn.v20190905', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r1i1p5f1.SImon.siv.gn.v20190905', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r4i1p5f1.SImon.siv.gn.v20190905', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r2i1p5f1.SImon.siv.gn.v20190905', 'CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.historical.r1i1p1f1.SImon.siv.gn.v20200919', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r5i1p1f1.SImon.siv.gn.v20190403', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r10i1p1f1.SImon.siv.gn.v20190403', 'CMIP6.CMIP.SNU.SAM0-UNICON.historical.r1i1p1f1.SImon.siv.gn.v20190323', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r7i1p1f1.SImon.siv.gn.v20190403', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r6i1p1f1.SImon.siv.gn.v20190403', 'CMIP6.CMIP.NASA-GISS.GISS-E2-1-H.historical.r3i1p1f1.SImon.siv.gn.v20190403', 'CMIP6.CMIP.N

In [12]:
iids

['CMIP6.CMIP.MIROC.MIROC6.historical.r6i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r3i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r4i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r10i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r9i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r5i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r1i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r8i1p1f1.SImon.siv.gn.v20190311',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r7i1p1f1.SImon.siv.gn.v20190311']

In [19]:
data.zstore.replace('gs://cmip6/', '').replace('/', '.')

'CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r3i1p1f1.SImon.siv.gn.v20200817.'